<a href="https://colab.research.google.com/github/Eliandry/Neural-networks/blob/main/guessWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import re
import numpy as np
from keras.layers import Dense,SimpleRNN,Input,Embedding
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [30]:
with open('/content/fan', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')

In [31]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

data = tokenizer.texts_to_sequences([texts])
# res = to_categorical(data[0], num_classes=maxWordsCount)
# print(res.shape)
res = np.array( data[0] )

inp_words = 2
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

[('мы', 35), ('стояли', 2), ('в', 125), ('местечке', 2), ('жизнь', 4), ('армейского', 1), ('офицера', 1), ('известна', 1), ('утром', 2), ('ученье', 1)]


In [32]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 256)            256000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              129000    
Total params: 434,280
Trainable params: 434,280
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=80)

Epoch 1/80
113/113 [==============================] - 2s 9ms/step - loss: 6.7366 - accuracy: 0.0280
Epoch 2/80
113/113 [==============================] - 1s 9ms/step - loss: 5.8406 - accuracy: 0.0515
Epoch 3/80
113/113 [==============================] - 1s 9ms/step - loss: 5.5997 - accuracy: 0.0833
Epoch 4/80
113/113 [==============================] - 1s 9ms/step - loss: 5.1906 - accuracy: 0.1115
Epoch 5/80
113/113 [==============================] - 1s 8ms/step - loss: 4.6076 - accuracy: 0.1689
Epoch 6/80
113/113 [==============================] - 1s 9ms/step - loss: 4.0503 - accuracy: 0.2549
Epoch 7/80
113/113 [==============================] - 1s 9ms/step - loss: 3.5470 - accuracy: 0.3382
Epoch 8/80
113/113 [==============================] - 1s 8ms/step - loss: 3.0103 - accuracy: 0.4095
Epoch 9/80
113/113 [==============================] - 1s 8ms/step - loss: 2.6375 - accuracy: 0.4736
Epoch 10/80
113/113 [==============================] - 1s 8ms/step - loss: 2.2743 - accuracy: 0.5271

In [45]:
def buildPhrase(texts, str_len=5):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        # x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        # inp = x.reshape(1, inp_words, maxWordsCount)
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [46]:
res = buildPhrase("кроме двух")
print(res)

кроме двух или трех коих большею частию
